# Summarization

### Loading Libraries

In [5]:
# Numerical 
import numpy as np
#
import pandas as pd

# Natural Language Toolkit
import nltk
from nltk.tokenize import sent_tokenize

# Datasets
from datasets import load_dataset
from datasets import load_metric

# Transformers
import transformers
from transformers import pipeline, set_seed
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Data Visualization
import matplotlib.pyplot as plt

# Progress bar & Iterations Management
from tqdm import tqdm

# PyTorch
import torch

# HuggingFace Hub
from huggingface_hub import notebook_login

### Retrieving Data

In [3]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks

from install import *
install_requirements(is_chapter6=True)

Cloning into 'notebooks'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 526 (delta 143), reused 135 (delta 126), pack-reused 353
Receiving objects: 100% (526/526), 28.62 MiB | 20.46 MiB/s, done.
Resolving deltas: 100% (250/250), done.
/Users/isisromero/Desktop/NLP_transformer/chap_06/notebooks
⏳ Installing base requirements ...


/Users/isisromero/anaconda3/envs/NLP_transformer/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [4]:
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Using transformers v4.40.1
Using datasets v2.19.0


In [12]:
# 
dataset = load_dataset("cnn_dailymail", "3.0.0")
print(f"Features: {dataset['train'].column_names}")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Features: ['article', 'highlights', 'id']
